In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
list_of_urls = [
    'http://www.boxofficemojo.com/yearly/chart/?yr=2015&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2014&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2013&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2012&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2011&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2010&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2009&p=.htm',
]

In [24]:
url = 'http://www.boxofficemojo.com/yearly/chart/?yr=2015&p=.htm'
response = requests.get(url)
soup = BeautifulSoup(response.text)

In [27]:
table = soup.find_all('table')[3]

In [28]:
movies = table.find_all('tr')

In [85]:
df = pd.DataFrame(columns=('title', 'studio', 'total_gross','theaters', 'opening','open_theaters', 'open', 'close', 'intlurl'))

In [97]:
testdata = movies[6].find_all('td')

In [98]:
myurl = testdata[1].find('a')['href']

In [99]:
myurl = 'http://www.boxofficemojo.com' + myurl

In [100]:
myurl = myurl.split('?')[0] + '?page=intl&' + myurl.split('?')[1]

In [101]:
myurl

'http://www.boxofficemojo.com/movies/?page=intl&id=starwars7.htm'

In [107]:
for row in movies[6:106]:
    row = row.find_all('td')
    row_s = pd.Series([item.get_text() for item in row][1:])
    intl = row[1].find('a')['href']
    intl = 'http://www.boxofficemojo.com' + intl
    intl = intl.split('?')[0] + '?page=intl&' + intl.split('?')[1]
    row_s = row_s.append(pd.Series(intl))
    row_s2 = row_s(index=df.columns)
print row_s2
#     df = df.append(row_s, ignore_index=True)
    

NameError: name 'row_s2' is not defined

In [59]:
df

,title,studio,total_gross,theaters,opening,open_theaters,open,close
0,Star Wars: The Force Awakens,BV,"$879,110,994","4,134","$247,966,675","4,134",12/18,-
1,Jurassic World,Uni.,"$652,270,625","4,291","$208,806,270","4,274",6/12,11/19
2,Avengers: Age of Ultron,BV,"$459,005,868","4,276","$191,271,109","4,276",5/1,10/8
3,Inside Out,BV,"$356,461,711","4,158","$90,440,272","3,946",6/19,12/10
4,Furious 7,Uni.,"$353,007,020","4,022","$147,187,040","4,004",4/3,7/24
5,Minions,Uni.,"$336,045,770","4,311","$115,718,405","4,301",7/10,12/17
6,The Hunger Games: Mockingjay - Part 2,LGF,"$280,302,457","4,175","$102,665,981","4,175",11/20,-
7,The Martian,Fox,"$227,616,581","3,854","$54,308,575","3,831",10/2,-
8,Cinderella (2015),BV,"$201,151,353","3,848","$67,877,361","3,845",3/13,9/17
9,Spectre,Sony,"$199,270,555","3,929","$70,403,148","3,929",11/6,-
